# 数据聚合与分组计算

对数据集进行**分组**并对各组应用一个**函数**(无论是聚合还是转换),这是数据分析工作中的重要环节。在将数据集准备好之后,通常的任务就是**计算分组统计**或**生成透视表**。pandas提供了一个灵活高效的**gruopby**功能,它使你能以一种自然的方式对数据集进行**切片**、**切块**、**摘要**等操作。

关系型数据库和SQL(Structured Query Language,结构化查询语言)能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。但是,像SQL这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到,由于Python和pandas强大的表达能力,我们可以执行复杂得多的分组运算(利用任何可以接受pandas对象或NumPy数组的函数)。

在本章中,你将会学到:
* 根据一个或多个键(可以是函数、数组或DataFrame列名)拆分pandas对象。
* 计算分组摘要统计,如计数、平均值、标准差,或用户自定义函数。
* 对DataFrame的列应用各种各样的函数。
* 应用组内转换或其他运算,如规格化、线性回归、排名或选取子集等。
* 计算透视表或交叉表。
* 执行分位数分析以及其他分组分析。

In [1]:
import numpy as np

import pandas as pd
from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def print_gb(gb):
    for n,g in gb:
        print n
        print g
        print '\n'

## GroupBy技术

### 分组运算（split-apply-combine）

分组运算（split-apply-combine）：
* 拆分：
    * 通过一个或多个键对原数据进行拆分到不同组中；
* 应用：
    * 在不同组上应用函数计算得到结果；
* 合并：
    * 将结果合并到最终的结果对象中；

下图很好的展示了该过程：
![分组计算](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/%E5%88%86%E7%BB%84%E8%AE%A1%E7%AE%97.png?raw=true)

### 分组键的可取情况

分组键可以有多种形式,且类型不必相同:
* 列表或数组,其长度与待分组的轴一样。
* 表示DataFrame某个列名的值。
* 字典或Series,给出待分组轴上的值与分组名之间的对应关系。
* 函数,用于处理轴索引或索引中的各个标签。
**注意**：后三种本质上是第一种的快捷方式，通过各种方式获取用于拆分对象的值，因此可以将这四种方式看做是如何获取用于拆分对象的值的四种方式即可，第一种是直接使用数组，第二种是取列名，第三种是映射关系，第四种是靠返回值；

注意：不管分组时表面上使用的是什么，最终都会转换成一个用于对应数据应该处于哪个分组的数组，数组上每个值，决定了相应位置的数据应该属于哪个分组；

### 分组示例

#### 使用Series做分组键 -- 例如df['key1']

In [3]:
df = DataFrame({'data1':[10,20,30,40,50],'data2':[40,50,60,70,80],
                'key1':['a','b','a','b','a'],'key2':['c','d','d','c','c']},
              index=['HL','LM','BL','JK','MP'])
df

,data1,data2,key1,key2
HL,10,40,a,c
LM,20,50,b,d
BL,30,60,a,d
JK,40,70,b,c
MP,50,80,a,c


In [4]:
# 对data1列数据按照key1分组并聚合计算平均值
df['data1'].groupby(df['key1']).mean() # 生成Series索引为key1的唯一值

key1
a    30
b    30
Name: data1, dtype: int64

In [5]:
# 对data1按照key1，key2分组并计算平均值
df['data1'].groupby([df['key1'],df['key2']]).mean() # 生成Series索引为key1，key2的唯一键组合

key1  key2
a     c       30
      d       30
b     c       40
      d       20
Name: data1, dtype: int64

#### 任意数组做分组键 -- 数组每个值对应同位置行的值，也就是强行有一种映射关系

In [6]:
arr = np.array(['aa','bb','cc','aa','cc'])
df['data1'].groupby(arr).mean()

aa    25
bb    20
cc    40
Name: data1, dtype: int64

#### 将列名(可以是字符串、数字或其他Python对象)用作分组键 -- 默认丢弃非数值组

In [7]:
df.groupby('key1').mean() # 使用列名作为分组键时不能针对某一列（Series）分组了就，因为Series没有该列名

,data1,data2
key1,,
a,30,60
b,30,60


#### groupby的size

In [8]:
df.groupby('key2').size()

key2
c    3
d    2
dtype: int64

### 对分组进行迭代

GroupBy对象支持迭代,可以产生一组二元元组(由分组名和数据块组成)。

#### 单键分组迭代

In [9]:
for name,group in df.groupby('key1'):
    print name
    print group
    print '\n'

a
    data1  data2 key1 key2
HL     10     40    a    c
BL     30     60    a    d
MP     50     80    a    c


b
    data1  data2 key1 key2
LM     20     50    b    d
JK     40     70    b    c




#### 多键分组迭代

In [10]:
for names,group in df.groupby(['key1','key2']):
    print names
    print group
    print '\n'

('a', 'c')
    data1  data2 key1 key2
HL     10     40    a    c
MP     50     80    a    c


('a', 'd')
    data1  data2 key1 key2
BL     30     60    a    d


('b', 'c')
    data1  data2 key1 key2
JK     40     70    b    c


('b', 'd')
    data1  data2 key1 key2
LM     20     50    b    d




#### 将分组结果转换为字典

In [11]:
group_dict = dict(list(df.groupby(['key1','key2'])))
for key in group_dict:
    print key
    print group_dict[key]
    print '\n'

('b', 'c')
    data1  data2 key1 key2
JK     40     70    b    c


('a', 'd')
    data1  data2 key1 key2
BL     30     60    a    d


('a', 'c')
    data1  data2 key1 key2
HL     10     40    a    c
MP     50     80    a    c


('b', 'd')
    data1  data2 key1 key2
LM     20     50    b    d




#### 在索引上分组 -- 指定axis=0

In [12]:
for name,group in df.groupby(['A','A','B','B','B'], axis=0):
    print name
    print group
    print '\n'

A
    data1  data2 key1 key2
HL     10     40    a    c
LM     20     50    b    d


B
    data1  data2 key1 key2
BL     30     60    a    d
JK     40     70    b    c
MP     50     80    a    c




### 选取一个或一组列 -- 可以直接对指定的列进行分组，或对分组结果取对应列

#### 对指定的列进行分组

In [13]:
df['data2'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     c       60
      d       60
b     c       70
      d       50
Name: data2, dtype: int64

#### 对分组结果取指定列 -- 这种方式是上一种方式的语法糖

In [14]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     c       60
      d       60
b     c       70
      d       50
Name: data2, dtype: int64

#### 注意下述两种写法的不同之处

In [15]:
df.groupby(['key1'])['data2'].mean()

key1
a    60
b    60
Name: data2, dtype: int64

In [16]:
df.groupby(['key1'])[['data2']].mean()

,data2
key1,
a,60
b,60


比较：
* \['data2'\]：
    * 结果为Series；
    * Name属性为对应取的列名；
    * **DataFrame['列名']**得到的是对应列的**Series**形式；
* \[\['data2'\]\]：
    * 结果为DataFrame；
    * 索引为分组键，列为对应取的列名；
    * **DataFrame[['列名']]**得到的是对应列+原索引组成的**DataFrame**形式；

In [17]:
df['key1'] # 获取原索引+该列数据的Series

HL    a
LM    b
BL    a
JK    b
MP    a
Name: key1, dtype: object

In [18]:
df[['key1']] # 获取原索引+该列的DataFrame

,key1
HL,a
LM,b
BL,a
JK,b
MP,a


### 通过字典或Series进行分组

In [19]:
df = DataFrame(np.random.randn(5,5),
              columns=['a','b','c','d','e'],
              index=['01','02','03','04','05'])
df.ix[1:4,1:4] = np.nan # 设置几个nan值
df

/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


,a,b,c,d,e
01,-0.138747,-1.808818,1.146726,0.565915,-0.305556
02,-1.224948,NaN,NaN,NaN,-1.231459
03,1.696655,NaN,NaN,NaN,0.155311
04,-1.055390,NaN,NaN,NaN,0.707520
05,-0.515779,-0.117837,0.795646,0.020056,-1.002590


#### 通过字典分组

In [20]:
dict_col = {'a':'A','b':'B','c':'A','d':'A','e':'B'}
df.groupby(dict_col, axis=1).mean()# 指定分组关系，等价于df.groupby(['A','B','A','A','B'], axis=1).mean()

,A,B
01,0.524631,-1.057187
02,-1.224948,-1.231459
03,1.696655,0.155311
04,-1.055390,0.707520
05,0.099974,-0.560213


#### 通过Series分组 -- 

In [21]:
series_col = Series({'a':'A','b':'B','c':'A','d':'A','e':'B'}) # 长度不一定要一致的
df.groupby(series_col, axis=1).mean()

,A,B
01,0.524631,-1.057187
02,-1.224948,-1.231459
03,1.696655,0.155311
04,-1.055390,0.707520
05,0.099974,-0.560213


### 通过函数分组

相较于字典或Series,Python函数在定义分组映射关系时可以更
有创意且更为抽象。任何被当做分组键的函数都会在各个索引值上被
调用一次,其返回值就会被用作分组名称。

#### 纯函数分组

In [22]:
df = DataFrame({'grade':[67,54,47,82,66]}, index=['Jack Jr.','Murphy','Mark Jr.','Lily','John Jr.'])
df.groupby(lambda name:'Jr.' in name).mean() # 根据名称中是否存在Jr.进行分组统计分数平均值

,grade
False,68
True,60


#### 函数混合其他分组 -- 先使用函数分为True，False两组，再根据数组继续细分

In [23]:
df.groupby([lambda name:'Jr.' in name,['1','2','2','1','2']]).mean() # 函数混合数组

grade
False 1   82.0
      2   54.0
True  1   67.0
      2   56.5

### 根据索引级别分组 -- 直接通过level参数指定分组级别即可

In [24]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],[1, 3, 5, 1, 3]], names=['cty', 'tenor'])
df = DataFrame(np.random.randn(4, 5), columns=columns)
df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.807422 -0.368774  2.158580  0.609192  0.831243
1     -0.919084  0.973250  1.007917  1.204838 -0.451000
2     -1.838745 -0.623581  2.138910 -0.064356  1.389326
3     -1.248318  1.571616 -0.522442  0.190537 -0.927206

In [25]:
df.groupby(level='cty', axis=1).mean() # 按照cty分组，也就是最外层索引

cty,JP,US
0,0.720217,0.327461
1,0.376919,0.354028
2,0.662485,-0.107806
3,-0.368334,-0.066382


In [26]:
df.groupby(level=1, axis=1).mean() # 按照最内层索引分组

tenor,1,3,5
0,-0.099115,0.231234,2.158580
1,0.142877,0.261125,1.007917
2,-0.951550,0.382873,2.138910
3,-0.528891,0.322205,-0.522442


## 数据聚合

对于**聚合**,我指的是任何能够从**数组**产生**标量值**的**数据转换**过
程。之前的例子中我已经用过一些,比如mean、count、min以及sum等。
你可能想知道在GroupBy对象上调用mean()时究竟发生了什么。许多
常见的聚合运算都有就地计算数据集统计信息的优化
实现。然而,并不是只能使用这些方法。你可以使用**自己发明**的**聚合运
算**,还可以调用分组对象上**已经定义**好的任何**方法**。

### 一般聚合方法

In [27]:
gb = DataFrame({'data1':[10,20,30,40,50],'data2':[40,50,60,70,80],
                'key1':['a','b','a','b','a'],'key2':['c','d','d','c','c']},
              index=['HL','LM','BL','JK','MP']).groupby('key1')

for name,group in gb:
    print name
    print group
    print '\n'

a
    data1  data2 key1 key2
HL     10     40    a    c
BL     30     60    a    d
MP     50     80    a    c


b
    data1  data2 key1 key2
LM     20     50    b    d
JK     40     70    b    c




#### quantile -- 分位数 默认丢弃非数值列

In [28]:
gb.quantile(0.9)

0.9,data1,data2
key1,,
a,46.0,76.0
b,38.0,68.0


#### 使用自定义聚合方法 -- agg(callable, string, dictionary, or list of string/callables)

In [29]:
def func(x):
    return x.max() - x.min()

gb.agg(func) # 应用于每一个分组

,data1,data2
key1,,
a,40,40
b,20,20


注意: 可能你已经注意到了,自定义聚合函数要比内置的那些
经过优化的函数慢得多。这是因为在构造**中间分组数据块**时存在非常
大的开销(**函数调用**、**数据重排**等)。

#### GroupBy内置的可用聚合方法

* count：分组中非NaN的值的个数；
* sum：分组中非NaN值的和；
* mean：分组中非NaN值的平均值；
* median：分组中非NaN值的算数中位数；
* std,var：分组中非NaN值的无偏（分母为n-1，矫正过）标准差/方差；
* min,max：分组中非NaN值的最小/最大值；
* prod：分组中非NaN值的积；
* first,last：分组第一个/最后一个非NaN的值；

### 对比agg，aggregate，apply

In [30]:
def test(x):
    print type(x)
    print x
    print '\n'
    return 2

#### agg

In [31]:
gb.agg(test)

<class 'pandas.core.series.Series'>
HL    10
BL    30
MP    50
Name: data1, dtype: int64


<class 'pandas.core.series.Series'>
LM    20
JK    40
Name: data1, dtype: int64


<class 'pandas.core.series.Series'>
HL    40
BL    60
MP    80
Name: data2, dtype: int64


<class 'pandas.core.series.Series'>
LM    50
JK    70
Name: data2, dtype: int64


<class 'pandas.core.series.Series'>
HL    c
BL    d
MP    c
Name: key2, dtype: object


<class 'pandas.core.series.Series'>
LM    d
JK    c
Name: key2, dtype: object




,data1,data2,key2
key1,,,
a,2,2,2
b,2,2,2


agg：
* 作用于每一组的每一个列（也就是Series）上；
* 结果是聚合结果组成的DataFrame

#### aggregate

In [32]:
gb.aggregate(test)

<class 'pandas.core.series.Series'>
HL    10
BL    30
MP    50
Name: data1, dtype: int64


<class 'pandas.core.series.Series'>
LM    20
JK    40
Name: data1, dtype: int64


<class 'pandas.core.series.Series'>
HL    40
BL    60
MP    80
Name: data2, dtype: int64


<class 'pandas.core.series.Series'>
LM    50
JK    70
Name: data2, dtype: int64


<class 'pandas.core.series.Series'>
HL    c
BL    d
MP    c
Name: key2, dtype: object


<class 'pandas.core.series.Series'>
LM    d
JK    c
Name: key2, dtype: object




,data1,data2,key2
key1,,,
a,2,2,2
b,2,2,2


同上

#### apply

In [33]:
gb.apply(test)

<class 'pandas.core.frame.DataFrame'>
    data1  data2 key2
HL     10     40    c
BL     30     60    d
MP     50     80    c


<class 'pandas.core.frame.DataFrame'>
    data1  data2 key2
HL     10     40    c
BL     30     60    d
MP     50     80    c


<class 'pandas.core.frame.DataFrame'>
    data1  data2 key2
LM     20     50    d
JK     40     70    c




key1
a    2
b    2
dtype: int64

In [34]:
gb.apply(lambda x:x.min())

,data1,data2,key2
key1,,,
a,10,40,c
b,20,50,c


apply：
* 作用于每一个分组；
* 聚合结果是每一个分组的结果组成的Series；

但是为什么有三次循环呢？？

#### 总结

apply是作用于每一个分组上的，而agg，aggregate是作用于每一个分组的每一列上；

### 小费示例

In [35]:
# step1 加载数据
tips = pd.read_csv('https://raw.githubusercontent.com/NemoHoHaloAi/pydata-book/2nd-edition/examples/tips.csv')
tips.head(5)

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [36]:
# step2 增加小费所占比例列
tips['tip_proportion'] = tips['tip'] / tips['total_bill']
tips.head(5)

,total_bill,tip,smoker,day,time,size,tip_proportion
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [37]:
# step3 增加sex列，这个主要因为目前的数据没有这一列。。。
tips['sex'] = ['male' if x%3==0 else 'female' for x in np.arange(len(tips))]
tips.head(5)

,total_bill,tip,smoker,day,time,size,tip_proportion,sex
0,16.99,1.01,No,Sun,Dinner,2,0.059447,male
1,10.34,1.66,No,Sun,Dinner,3,0.160542,female
2,21.01,3.50,No,Sun,Dinner,3,0.166587,female
3,23.68,3.31,No,Sun,Dinner,2,0.139780,male
4,24.59,3.61,No,Sun,Dinner,4,0.146808,female


### 面向列的多函数应用

对Series或DataFrame列的**聚合运算**其实就是使用
**aggregate**(使用自定义函数)或调用诸如**mean**、**std**之类的方法。然而,
你可能希望对**不同的列**使用**不同的聚合函数**,或**一次应用多个函数**。

In [38]:
gb_sex_smoker = tips.groupby(['sex','smoker'])
print_gb(gb_sex_smoker)

('female', 'No')
     total_bill   tip smoker   day    time  size  tip_proportion     sex
1         10.34  1.66     No   Sun  Dinner     3        0.160542  female
2         21.01  3.50     No   Sun  Dinner     3        0.166587  female
4         24.59  3.61     No   Sun  Dinner     4        0.146808  female
5         25.29  4.71     No   Sun  Dinner     4        0.186240  female
7         26.88  3.12     No   Sun  Dinner     4        0.116071  female
8         15.04  1.96     No   Sun  Dinner     2        0.130319  female
10        10.27  1.71     No   Sun  Dinner     2        0.166504  female
11        35.26  5.00     No   Sun  Dinner     4        0.141804  female
13        18.43  3.00     No   Sun  Dinner     4        0.162778  female
14        14.83  3.02     No   Sun  Dinner     2        0.203641  female
16        10.33  1.67     No   Sun  Dinner     3        0.161665  female
17        16.29  3.71     No   Sun  Dinner     3        0.227747  female
19        20.65  3.35     No   Sat

#### 应用一个函数

In [39]:
gb_sex_smoker['tip_proportion'].agg('mean') # 对于tips_proportion进行平均值聚合计算

sex     smoker
female  No        0.162939
        Yes       0.171521
male    No        0.151813
        Yes       0.148060
Name: tip_proportion, dtype: float64

#### 应用多个函数

In [40]:
def test(x):
    return x.max() - x.min()

gb_sex_smoker['tip_proportion'].agg(['mean', test, np.std]) # 同时应用平均值，自定义函数，np内置标准差函数

mean      test       std
sex    smoker                              
female No      0.162939  0.219028  0.037195
       Yes     0.171521  0.644685  0.095908
male   No      0.151813  0.195876  0.044509
       Yes     0.148060  0.244897  0.059199

#### 针对单个列分别应用不同函数 -- 类似维护一个该列名称与实际聚合操作的映射表

In [47]:
# 对tip_proportion应用平均值，np.max和lambda表达式
gb_sex_smoker['tip_proportion'].agg([('平均小费','mean'), ('最大小费',np.max), ('平均小费/10', lambda x:x.mean()/10)])

平均小费      最大小费   平均小费/10
sex    smoker                              
female No      0.162939  0.291990  0.016294
       Yes     0.171521  0.710345  0.017152
male   No      0.151813  0.252672  0.015181
       Yes     0.148060  0.280535  0.014806

#### 针对多个列同时应用多个函数 -- 其实就是将多个不同的函数同时应用到每个列，然后将结果concat起来得到一个具有层次化索引的结果

In [50]:
gb_sex_smoker['tip_proportion','total_bill'].agg([('平均','mean'), ('最大',np.max)])

tip_proportion           total_bill       
                          平均        最大         平均     最大
sex    smoker                                           
female No           0.162939  0.291990  19.860196  48.33
       Yes          0.171521  0.710345  20.183333  50.81
male   No           0.151813  0.252672  17.789592  48.17
       Yes          0.148060  0.280535  21.798182  44.30

#### 对不同的列分别应用多个不同的函数 -- 需要传入表示列与聚合操作对应关系的字典

In [55]:
gb_sex_smoker.agg({'tip':('mean','min'), 'total_bill':('max',np.std,np.var)})

total_bill                              tip      
                     max        std         var      mean   min
sex    smoker                                                  
female No          48.33   8.460528   71.580533  3.180882  1.25
       Yes         50.81  10.475436  109.734768  3.066167  1.00
male   No          48.17   7.707749   59.409391  2.598367  1.00
       Yes         44.30   8.594847   73.871397  2.904242  1.17

### 以”无索引“的形式返回聚合数据 -- 默认索引由分类的属性值决定，比如上述的male或female，或层次化等

In [67]:
print_gb(tips[:10].groupby('sex', as_index=False)) # 指定不使用分类列属性作为索引

female
   total_bill   tip smoker  day    time  size  tip_proportion     sex
1       10.34  1.66     No  Sun  Dinner     3        0.160542  female
2       21.01  3.50     No  Sun  Dinner     3        0.166587  female
4       24.59  3.61     No  Sun  Dinner     4        0.146808  female
5       25.29  4.71     No  Sun  Dinner     4        0.186240  female
7       26.88  3.12     No  Sun  Dinner     4        0.116071  female
8       15.04  1.96     No  Sun  Dinner     2        0.130319  female


male
   total_bill   tip smoker  day    time  size  tip_proportion   sex
0       16.99  1.01     No  Sun  Dinner     2        0.059447  male
3       23.68  3.31     No  Sun  Dinner     2        0.139780  male
6        8.77  2.00     No  Sun  Dinner     2        0.228050  male
9       14.78  3.23     No  Sun  Dinner     2        0.218539  male




## 分组级运算和转换

## 透视表和交叉表

## 示例：2012联邦